<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Subroutines**

In [ ]:
!pip install cirq --quiet
import cirq
import sympy
from cirq.contrib.svg import SVGCircuit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # %matplotlib inline
print(cirq.__version__)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_041.JPG)

https://pennylane.ai/qml/demos_qml.html

https://www.nature.com/articles/npjqi201523#Sec12

https://qiskit.org/textbook/ch-appendix/linear_algebra.html 

### <font color="blue">**Encoding Classical Information**

https://youtu.be/LHzWAyT5rMI

Basis Encoding

Amplitude Encoding

Hamiltonian Encoding

### **<font color="red">Quantum Walk**

https://towardsdatascience.com/creating-a-quantum-walk-with-a-quantum-coin-692dcfa30d90

http://quantum-technologies.iap.uni-bonn.de/de/quantum-walks.html

https://quantumai.google/cirq/tutorials/quantum_walks

https://medium.com/qiskit/studying-quantum-walks-on-near-term-quantum-computers-f60fd2395f04

* [Quantum walks](https://en.m.wikipedia.org/wiki/Quantum_walk) are quantum analogues of classical random walks. In contrast to the classical random walk, where the walker occupies definite states and the randomness arises due to stochastic transitions between states, in quantum walks randomness arises through: 

  (1) quantum superposition of states, 
  
  (2) non-random, reversible unitary evolution and 
  
  (3) collapse of the wave function due to state measurements.



As with classical random walks, quantum walks admit formulations in both discrete time and continuous time.

* Quantum walks are motivated by the widespread use of classical random walks in the design of randomized algorithms, and are part of several quantum algorithms. For some oracular problems, quantum walks provide an exponential speedup over any classical algorithm.

* Quantum walks also give polynomial speedups over classical algorithms for many practical problems, such as the element distinctness problem, the triangle finding problem, and evaluating NAND trees. The well-known Grover search algorithm can also be viewed as a quantum walk algorithm

* Quantum walks exhibit very different features from classical random walks. In particular, they do not converge to [limiting distributions](https://en.m.wikipedia.org/wiki/Asymptotic_distribution) and due to the power of quantum interference they may spread significantly faster or slower than their classical equivalents.

*Continuous-time quantum walks*

* [Continuous-time quantum walks](https://en.m.wikipedia.org/wiki/Continuous-time_quantum_walk) arise when one replaces the continuum spatial domain in the Schrödinger equation with a discrete set.

* Under particular conditions, continuous-time quantum walks can provide a model for universal quantum computation.

###### **<font color="blue">Exkurs: Qumodes, Qudits and Qutrits</font>**

**<font color="blue">Qumodes</font>**

(CV systems / continuous spectra in infinite-dimensional Hilbert space)

https://strawberryfields.ai/photonics/concepts/photonics.html

**<font color="blue">Qudits and Qutrits</font>**

https://en.m.wikipedia.org/wiki/Qutrit

https://en.m.wikipedia.org/wiki/Qubit#Qudits_and_qutrits

### <font color="blue">**Pending Algorithms**

#### **<font color="blue">Simple Addition with Qubits**

**Task 1: Addition of 1 + 0 + 0 = 1 (01 in binary): We flip q0 and q1 from 0 to 1 with Pauli-X**

**Task 2: Addition of 1 + 1 + 0 = 2 (10 in binary): We flip q0 and q1 from 0 to 1 with Pauli-X**

**Task 3: Addition of 1 + 1 + 1 = 3 (11 in binary): We flip q0, q1 and q2 from 0 to 1 with Pauli-X**

https://qiskit.org/textbook/ch-states/atoms-computation.html#adder

0+0 = 00 (in decimal, this is 0+0=0)

0+1 = 01 (in decimal, this is 0+1=1)

1+0 = 01 (in decimal, this is 1+0=1)

1+1 = 10 (in decimal, this is 1+1=2)

This is called a half adder. If our computer can implement this, and if it can chain many of them together, it can add anything.



In [ ]:
# Create Circuit object
circuit = cirq.Circuit()

# Define 2 qubits - They start with 0 (if you measure no, you'll see a 0 in both)
(q0, q1, q2, q3) = cirq.LineQubit.range(4)



## Set up calculation:

# We flip q0 from 0 to 1 with Pauli-X 
circuit.append([cirq.X(q0)]) 

# We flip q1 from 0 to 1 with Pauli-X 
#circuit.append([cirq.X(q1)]) 

# We flip q2 from 0 to 1 with Pauli-X 
#circuit.append([cirq.X(q2)])



# CNOT (Controlled Pauli X) for input q2 and target q0 (0 is same, 1 if different for target)
# since q2 is 0 and q0 is 1, result will stay 1
circuit.append([cirq.CNOT(q0, q2)])

# CNOT (Controlled Pauli X) for input q2 and target q1 (0 is same, 1 if different for target)
# since q2 is 0 and q1 is 1, result will stay 1
circuit.append([cirq.CNOT(q1, q2)])

# Toffoli (CCNOT) gate on q3 (if both inputs are equal, write 1, else 0) (perform a NOT on target qubit when both controls are in state 1)
circuit.append([cirq.CCX(q0, q1, q3)])

# Measure output quantum bits q2 and q3
circuit.append([cirq.measure(q2), cirq.measure(q3)])

# See what circuit has been produced (optional)
print(circuit)

                  ┌──┐
0: ───X───@────────@─────────
          │        │
1: ───────┼───@────@─────────
          │   │    │
2: ───────X───X────┼M────────
                   │
3: ────────────────X─────M───
                  └──┘


In [ ]:
# Step 6: Perform simulation of the circuit and read out result
sim = cirq.Simulator()
results = sim.run(circuit, repetitions=10)
print(results)

2=1111111111
3=0000000000


> Read from rightmost qubit first (q3) and then next left (q2): result is 10 (binary) = 2 in decimal

#### **<font color="red">Basic QFT-based Subroutines (Deutsch, Bernstein, Simon)**

###### **<font color="blue">Deutsch–Jozsa</font>**

https://richardwang-qm.medium.com/cirq-programming-3-the-deutsch-jozsa-bdc31982f187

https://jonathan-hui.medium.com/qc-quantum-algorithm-with-an-example-cf22c0b1ec31

Allgemein Oracel: https://towardsdatascience.com/the-quantum-oracle-demystified-65e8ffebd5d5

> Constant or balanced?

* nutzt Quantum Fourier Transform

There are four different possible functions:
* Function f_0 always returns 0.
* Function f_1 returns 0 if the input is 0 and it returns 1 if the input is 1.
* Function f_2 returns 1 if the input is 0 and it returns 0 if the input is 1.
* Function f_3 always returns 1.

Task: 
* Send a number of several numbers to determine, if the function is a constant or a balanced?
* f_0 und f_3 constant, f_1 und f_2 balanced
* classical needs at least 2 numbers (0,1), quantum needs just 1 number to solve this

https://towardsdatascience.com/how-to-solve-a-problem-with-quantum-computing-4b04f3f882d9

https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html

https://medium.com/quantum-untangled/quantum-untangled-quantum-computing-deutsch-jozsa-algorithm-guide-untangled-27adf4205913

https://medium.com/quantum-untangled/the-deutsch-jozsa-algorithm-math-circuits-and-code-quantum-algorithms-untangled-f3b28be4cfd3

https://en.m.wikipedia.org/wiki/Deutsch–Jozsa_algorithm

###### **<font color="blue">Bernstein–Vazirani algorithm</font>**

> What is a string?

https://en.m.wikipedia.org/wiki/Bernstein–Vazirani_algorithm

###### **<font color="blue">Simon's problem</font>**

https://jonathan-hui.medium.com/qc-simons-algorithm-be570a40f6de

> 1-1 or 2-1 ?

https://en.m.wikipedia.org/wiki/Simon%27s_problem

###### **<font color="black">Boson sampling problem ('Aaronson–Arkiphov Scheme')</font>**

https://www.nature.com/articles/d41586-020-03434-7

The Hefei team, led by Pan and Chao-Yang Lu, chose a different problem for its demonstration, called boson sampling. It was devised in 2011 by two computer scientists, Scott Aaronson and Alex Arkhipov4, then at the Massachusetts Institute of Technology in Cambridge. It entails calculating the probability distribution of many bosons — a category of fundamental particle that includes photons — whose quantum waves interfere with one another in a way that essentially randomizes the position of the particles. The probability of detecting a boson at a given position can be calculated from an equation in many unknowns.

###### **<font color="black">Quantum Minimization Algorithm (QMA)**

* Quantum algorithm for solving discrete functions minimization problems was introduced in. It is based on Grover’s search and can be represented in terms of quantum iterations circuits.

* Such algorithm allows us to find global optimum as it is always searching through all possible inputs states

* Christoph, D. and Hoyer, P. "A quantum algorithm for finding the minimum", arXiv:quant-ph/9607014 (1-2, 1996) https://arxiv.org/abs/quant-ph/9607014

* https://arxiv.org/pdf/1906.10175.pdf

#### <font color="black">**Hidden Subgroup Problem (HSP)**

Shor’s approach to integer factorisation is based on reducing the task to a special case of a mathematical problem known as the hidden subgroup problem (HSP), then giving an efficient quantum algorithm for this problem.

https://arxiv.org/pdf/1511.04206.pdf

#### <font color="black">**More Quantum Linear Algebra & Optimization**

###### **<font color="blue">Quantum Power Iteration & Inverse Iteration**

* used for Eigenvalue calculation in large matrices

* [Power iteration (Eigenvalues)](https://en.m.wikipedia.org/wiki/Power_iteration) und [Inverse Iteration (Eigenvector)](https://en.m.wikipedia.org/wiki/Inverse_iteration)

* [Quantum Power Method by a Superposition of Time-Evolved States](https://arxiv.org/abs/2008.03661)

###### **<font color="blue">Quantum Multiplication</font>**

* **Boost Multiplication of Karatsuba with Quantum Computers**: In his new paper, Gidney describes a quantum way of implementing Karatsuba multiplication that doesn’t impose huge memory costs. Instead of generating intermediate values to produce a final value, he uses a method called “tail call optimization” to directly mutate inputs to outputs. 

* This allows the algorithm to avoid creating intermediate information that a quantum computer can never discard. “He gets rid of the issue of having to deal with extra qubits by not having extra qubits,” said Thomas Wong, a quantum information scientist at Creighton University.Gidney expects that his method will work for adapting many classical recursive algorithms to quantum computers. 

[A New Approach to Multiplication Opens the Door to Better Quantum Computers](https://www.quantamagazine.org/a-new-approach-to-multiplication-opens-the-door-to-better-quantum-computers-20190424/)

###### **<font color="blue">Quantum Principal Component Analysis</blue>**

S. Lloyd, M. Mohseni, and P. Rebentrost, Nature Physics 10,
631 (2014)

https://arxiv.org/abs/1307.0401

###### **<font color="blue">Quantum Linear Regression</blue>**

https://www.quantamagazine.org/new-algorithm-breaks-speed-limit-for-solving-linear-equations-20210308/

https://www.quantamagazine.org/teenager-finds-classical-alternative-to-quantum-recommendation-algorithm-20180731/

https://www.quantamagazine.org/mathematicians-inch-closer-to-matrix-multiplication-goal-20210323/

https://www.quantamagazine.org/a-new-approach-to-multiplication-opens-the-door-to-better-quantum-computers-20190424/

M. Schuld, I. Sinayskiy, and F. Petruccione, arXiv preprint
arXiv:1601.07823 (2016). https://arxiv.org/abs/1601.07823

#### **<font color="black">Quantum Simulation**

... (Monte Carlo, Quantum Chemistry, Materials Science and more)


###### **<font color="blue">Quantum Simulators**

https://en.wikipedia.org/wiki/Quantum_chemistry

https://en.m.wikipedia.org/wiki/Quantum_simulator

https://de.wikipedia.org/wiki/Quantensimulation

quantum chemistry: https://www.youtube.com/watch?v=2XEjrwWhr88

How is Quantum Phase Estimation applied to Quantum Chemistry calculations: in Quantum Chemistry you are interested in dynamics = time evolution. QPE is helping us to ...so Hamiltonian dynamics is unitary, and so you can use that unitary that comes from your Hamiltonian to do simulations in chemistry.

* Many important problems in physics, especially [low-temperature physics](https://en.m.wikipedia.org/wiki/Thermodynamics) and [many-body physics](https://en.m.wikipedia.org/wiki/Many-body_theory), remain poorly understood because the underlying quantum mechanics is vastly complex. See also [Condensed_matter_physics](https://en.m.wikipedia.org/wiki/Condensed_matter_physics) (incl. crystallography etc)

* Conventional computers, including supercomputers, are inadequate for simulating quantum systems with as few as 30 particles. Better computational tools are needed to understand and **rationally design materials whose properties are believed to depend on the collective quantum behavior of hundreds of particles**.

* Quantum simulators provide an alternative route to understanding the properties of these systems. These simulators create clean realizations of specific systems of interest, which allows precise realizations of their properties. Precise control over and broad tunability of parameters of the system allows the influence of various parameters to be cleanly disentangled.

* Quantum simulators can solve problems which are difficult to simulate on classical computers **because they directly exploit quantum properties of real particles**. In particular, they exploit a property of quantum mechanics called superposition, wherein a quantum particle is made to be in two distinct states at the same time, for example, aligned and anti-aligned with an external magnetic field. Crucially, simulators also take advantage of a second quantum property called entanglement, allowing the behavior of even physically well separated particles to be correlated.



###### **<font color="blue">Stochastic Processes and Monte Carlo Simulations**

https://en.m.wikipedia.org/wiki/Quantum_Monte_Carlo

https://en.wikipedia.org/wiki/Quantum_Markov_chain

https://www.nature.com/articles/s41534-021-00459-2

Quantum Gaussian Process Algorithm (QGPA): https://arxiv.org/pdf/1803.10520.pdf

###### **<font color="blue">Hybrid: Variational quantum eigensolver (i.e. Protein Folding)**

The VQE algorithm applies classical optimization to minimize the energy expectation of an ansatz state to find the ground state energy of a molecule. This can also be extended to find excited energies of molecules.

*Optimization: simulations for material science and quantum chemistry, factoring numbers*

* Known as variational quantum algorithms, they use the quantum boxes to manipulate quantum systems while shifting much of the work load to classical computers to let them do what they currently do best: solve optimization problems.

* "With variational quantum algorithms, we get the best of both worlds. We can harness the power of quantum computers for tasks that classical computers can't do easily, then use classical computers to compliment the computational power of quantum devices."

* The algorithms are called variational because the optimization process varies the algorithm on the fly, as a kind of machine learning. It changes parameters and logic gates to minimize a cost function, which is a mathematical expression that measures how well the algorithm has performed the task. The problem is solved when the cost function reaches its lowest possible value. 

* In an iterative function in the variational quantum algorithm, the quantum computer estimates the cost function, then passes that result back to the classical computer. The classical computer then adjusts the input parameters and sends them to the quantum computer, which runs the optimization again. 



https://phys-org.cdn.ampproject.org/c/s/phys.org/news/2021-08-algorithms-small-noisy-quantum-viable.amp

https://www.nature.com/articles/s42254-021-00348-9

*Resource-efficient quantum algorithm for protein folding*

* Rather than spend computational resources checking each possible fold of a protein, the quantum approach encodes the superposition of all physically meaningful ways of folding the protein into a model Hamiltonian. Then it samples these combinations statistically to find the series of folds that are the most stable.

* “Everything in our body wants to be in the minimum free-energy configuration,” said Barkoutsos. “It is the most stable. And we generally say in nature — in our bodies — the most stable is the winning configuration.”
With this approach, the researchers simulated the folding of a 10-amino acid chain called Angiotensin with a 22 qubit quantum simulator, and a seven-amino acid neuropeptide on a 20-qubit quantum computer.
Multipurpose, open-source code

* This work relied on a modified Variational Quantum Eigensolver (VQE) algorithm, a major optimization tool in the quantum arsenal, drawn from the Qiskit Application Module.

* To suit their needs, the team modified the VQE to sample only the parts of the data relevant to the problem, producing a tool they called a Conditional Value-at-Risk (CVaR) VQE. In line with IBM Quantum’s open-source ethos, this tool is now available to the Qiskit community as of the latest Qiskit Nature release.

https://medium.com/qiskit/a-novel-quantum-algorithm-for-protein-folding-paving-the-way-toward-resolving-one-of-the-biggest-861112139ff0

https://www.nature.com/articles/s41534-021-00368-4

#### **<font color="black">Computing Knot Invariants**

https://en.m.wikipedia.org/wiki/Jones_polynomial

Witten had shown that the Chern-Simons topological quantum field theory (TQFT) can be solved in terms of Jones polynomials. A quantum computer can simulate a TQFT, and thereby approximate the Jones polynomial, which as far as we know, is hard to compute classically in the worst-case scenario.